# IBM Watson Annotator for Clinical Data Notebook

### API Index:

* [Analyze](#analyze)
* [Profiles](#profiles)
* [Annotator Flows](#flows)
* [Annotators](#annotators)
* [Cartridges](#cartridges)
* [Health Check](#health_check)
* [User Data](#user_data)

## Setup

In [ ]:
# Run variables
APIKEY = "<YOUR_APIKEY_HERE>" # DO NOT COMMIT THE KEY
VERSION = "2020-09-03"
URL = "https://us-east.wh-acd.cloud.ibm.com/wh-acd/api"
# The next 3 parameters are for the Cartridge Deployment APIs
CARTRIDGE_ZIP = "<CARTRIDGE_ZIP_LOCATION_HERE>"
CARTRIDGE_ID = "<CARTRIDGE_ID_HERE>"
CARTRIDGES_CONTENT_TYPE = "application/octet-stream" # SELECT ONE CONTENT TYPE
#CARTRIDGES_CONTENT_TYPE = "multipart/form-data"

In [ ]:
from ibm_whcs_sdk import annotator_for_clinical_data as acd
from ibm_cloud_sdk_core.authenticators.iam_authenticator import IAMAuthenticator
import time

# Instantiate service instance
# Replace {version}, {apikey}, and {url} below
service = acd.AnnotatorForClinicalDataV1 (
    authenticator=IAMAuthenticator(apikey=APIKEY),
    version=VERSION
)
service.set_service_url(URL)


---
<a id='analyze'></a>

# Analyze APIs

### Analyze with dynamic annotator flow
Annotator flow is defined within the request body - requires json content-type


In [ ]:
# POST /analyze

text = "Patient has lung cancer, but did not smoke. She may consider chemotherapy as part of a treatment plan."

try:
    anno_cd = acd.Annotator(name="concept_detection", parameters = {"include_optional_fields": ["medical_codes", "source_vocabularies"]})
    anno_neg = acd.Annotator(name="negation")
    flow_arr = [
        acd.FlowEntry(annotator=anno_cd),
        acd.FlowEntry(annotator=anno_neg)
    ]

    flow = acd.Flow(elements=flow_arr, async_=False)
    
    resp = service.analyze(text, flow)
    concepts = resp.concepts
    
    for concept in concepts:
        print("Type: ", concept.type, "~ Name: ", concept.preferred_name)
    
except acd.ACDException as ex:
   print ("Error Occurred:  Code ", ex.code, " Message ", ex.message, " CorrelationId ", ex.correlation_id)

### Analyze with persisted or out-of-the-box annotator flows

Request body may contain text/plain content-type when referencing a persisted flow in the analyze request

In [ ]:
# POST /analyze/{flow_id}

flowId = "wh_acd.ibm_clinical_insights_v1.0_standard_flow"
text = "Patient has lung cancer, but did not smoke. CT scan shows tumor in left lung. She may consider chemotherapy as part of a treatment plan."

try:
    response = service.analyze_with_flow(flowId, text)
    concepts = response.concepts
    print ("Concepts:")
    for concept in concepts:
        print("  Type: ", concept.type, "- Name: ", concept.preferred_name)
    attribute_values = response.attribute_values
    print ("Attributes:")
    for attribute_value in attribute_values:
        print("  Name: ", attribute_value.preferred_name, " ( Covered Text: ", attribute_value.covered_text, ")")
        if attribute_value.insight_model_data.diagnosis is not None:
            diag = attribute_value.insight_model_data.diagnosis
            print("    Diagnostic Scores:")
            if diag.usage is not None:
                diag_usage = diag.usage
                print("    - Explicit = ", diag_usage.explicit_score)
                print("    - Patient Reported = ", diag_usage.patient_reported_score)
                print("    - Discussed = ", diag_usage.discussed_score)
            print("    - Suspected = ", diag.suspected_score)
            print("    - Symptom = ", diag.symptom_score)
            print("    - Trauma = ", diag.trauma_score)
            print("    - Family History = ", diag.family_history_score)
        if attribute_value.insight_model_data.procedure is not None:
            proc = attribute_value.insight_model_data.procedure
            print("    Procedure Scores:")
            if proc.usage is not None:
                proc_usage = proc.usage
                print("    - Explicit = ", proc_usage.explicit_score)
                print("    - Pending = ", proc_usage.pending_score)
                print("    - Discussed = ", proc_usage.discussed_score)
            if proc.task is not None:
                proc_task = proc.task
                print("    - Therapeutic = ", proc_task.therapeutic_score)
                print("    - Diagnostic = ", proc_task.diagnostic_score)
            if proc.type is not None:
                proc_type = proc.type
                print("    - Procedure = ", proc_type.procedure_score)
                print("    - Device = ", proc_type.device_score)
        if attribute_value.insight_model_data.normality is not None:
            norm = attribute_value.insight_model_data.normality
            print("    Normality Scores:")
            if norm.usage is not None:
                norm_usage = norm.usage
                print("    - Normal = ", norm_usage.normal_score)
                print("    - Abnormal = ", norm_usage.abnormal_score)
                print("    - Unknown = ", norm_usage.unknown_score)
                print("    - NonFinding = ", norm_usage.non_finding_score)
            if norm.evidence is not None:
                for evi_entry in norm.evidence:
                    print("    - Evidence:")
                    print("         Begin = ", evi_entry.begin)
                    print("         End = ", evi_entry.end)
                    print("         Covered Text = ", evi_entry.covered_text)
        
except acd.ACDException as ex:
   print ("Error Occurred:  Code ", ex.code, " Message ", ex.message, " CorrelationId ", ex.correlation_id)

---
<a id='profiles'></a>

# Profile APIs

## Get Profiles


In [ ]:
# GET /profiles

try:
   resp = service.get_profiles()
   rslt = resp.result

   for id, profile in rslt.items():
       print("Profile:",id)

except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Get Profile


In [ ]:
# GET /profiles/{id}
try:
    resp = service.get_profile("wh_acd.ibm_clinical_insights_v1.0_profile")
    rslt = resp.result
    print("Id:", rslt['id'])
    print("Name:", rslt['name'])
    print("Description:", rslt['description'])
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Create Profile


In [ ]:
# POST /profiles

try:
    prof_anno = acd.Annotator(
        name="concept_detection",
        parameters = {"libraries": ["umls.latest"]})
    
    prof_anno_arr = [ prof_anno ]
    
    resp = service.create_profile (
        new_id="my_profile",
        new_name="my profile",
        new_description="my profile description",
        new_annotators=prof_anno_arr)
    
    print("Response Code:",resp.status_code)
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Update Profile


In [ ]:
# PUT /profiles/{id}

try:

    anno_cd = acd.Annotator("concept_detection", parameters = {"libraries": ["umls.latest"]})
    anno_attr = acd.Annotator("attribute_detection")
    
    prof_annos = [ anno_cd, anno_attr ]
    
    resp = service.update_profile(
        "my_profile",
        new_id="my_profile",
        new_name="my profile",
        new_description="my profile description - UPDATE",
        new_annotators=prof_annos)
    
    print("Response Code:",resp.status_code)
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Delete Profile


In [ ]:
# DELETE /profiles/{id}

try:
    
    resp = service.delete_profile("my_profile")
    print("Response Code:",resp.status_code)
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

---
<a id='flows'></a>

# Flow APIs

### Get Flows


In [ ]:
# GET /flows

try:
    resp = service.get_flows()
    rslt = resp.result
    for flow in rslt:
        print("Flow ID:",flow)
        
except acd.ACDException as ex:
    print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Get Flow


In [ ]:
# GET /flows/{id}

try:
    resp = service.get_flows_by_id("wh_acd.ibm_clinical_insights_v1.0_standard_flow")
    rslt = resp.result
    print("Id:", rslt['id'])
    print("Name:", rslt['name'])
    print("Desc:", rslt['description'])
    
    for anno_flow in rslt['annotatorFlows']:
        flow = anno_flow['flow']
        flow_entries = flow['elements']
        for flow_entry in flow_entries:
            print("Annotator: ", flow_entry['annotator']['name'])
            
except acd.ACDException as ex:
    print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Create Flow


In [ ]:
# POST /flows

try:
    anno_cd = acd.Annotator("concept_detection")
    anno_neg = acd.Annotator("negation")
    flow_entry_cd = acd.FlowEntry(annotator=anno_cd)
    flow_entry_neg = acd.FlowEntry(annotator=anno_neg)
    flow_entries = [ flow_entry_cd, flow_entry_neg ]
    flow = acd.Flow(elements=flow_entries, async_=False)
    anno_flow = acd.AnnotatorFlow(flow=flow)
    anno_flows = [anno_flow]
    resp = service.create_flows(
        new_id="my_flow",
        new_name="my flow",
        new_description="my flow description",
        new_annotator_flows=anno_flows)
    print("Response Code:",resp.status_code)
    print(resp.result)
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Update Flow


In [ ]:
# PUT /flows/{id}

try:
    
    anno_med = acd.Annotator(name="medication")
    anno_neg = acd.Annotator(name="negation")
    flow_entry_med = acd.FlowEntry(annotator=anno_med)
    flow_entry_neg = acd.FlowEntry(annotator=anno_neg)
    flow_entries = [ flow_entry_med, flow_entry_neg ]
    flow = acd.Flow(elements=flow_entries, async_=False)
    anno_flow = acd.AnnotatorFlow(flow=flow)
    anno_flows = [anno_flow]
    
    service.update_flows(
        "my_flow",
        new_id="my_flow",
        new_name="my flow name",
        new_description="my flow name - UPDATE",
        new_annotator_flows=anno_flows)
    print("Response Code:",resp.status_code)
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

### Delete Flow


In [ ]:
# DELETE /flows/{id}

try:
    resp = service.delete_flows("my_flow")
    print("Response Code:",resp.status_code)
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

---
<a id='annotators'></a>

# Annotators

### GET Annotators


In [ ]:
# GET /annotators

try:
    
    resp = service.get_annotators()
    rslt = resp.result
    for annotator in rslt:
        print(annotator)
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

### GET Annotator


In [ ]:
# GET /annotators/{id}

try:
    
    resp = service.get_annotators_by_id("concept_detection")
    rslt = resp.result
    print("Description:",rslt['description'])
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

---
<a id='health_check'></a>

# Health Check Status

In [ ]:
# GET /status/health_check

try:
    resp = service.get_health_check_status()
    print("Response Code:",resp.status_code)
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

----
<a id='cartridges'></a>

# Cartridge Deployment

## Deploy New Cartridge - POST /cartridges

In [ ]:
# Deploy new cartridge
try:
    f = open(CARTRIDGE_ZIP, "rb")
    # "multipart/form-data" is also supported for archive_file_content_type
    resp = service.cartridges_post_multipart(archive_file=f, archive_file_content_type=CARTRIDGES_CONTENT_TYPE)
    print(resp.status_code)
    time.sleep(5) # wait for async cartridge deploy completion
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

## Get List of Deployed Cartridges for the given tenant - GET /cartridges

In [ ]:
# Get list of deployed cartridges available to the requesting tenant
try:
    resp = service.cartridges_get()
    rslt = resp.result
    for cartridge in rslt['cartridges']:
        print(cartridge['id'])
        print("status: ",cartridge['status'])
        print("\n")
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

## Get Status of Specific Deployed Cartridge - GET /cartridges/{id}

In [ ]:
# Get details of specific cartridge deployment
try:
    resp = service.cartridges_get_id("acd_nifi_v1.0")
    rslt = resp.result
    print("id: ", rslt['id'])
    print("status: ", rslt['status'])
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

## Redeploy Cartridge - PUT /cartridges

In [ ]:
# Redeploy a cartridge

try:
    f = open(CARTRIDGE_ZIP, "rb")
    # "multipart/form-data" is also supported for archive_file_content_type
    resp = service.cartridges_put_multipart(archive_file=f, archive_file_content_type=CARTRIDGES_CONTENT_TYPE)
    print(resp.status_code)
    time.sleep(5) # wait for async cartridge deploy completion
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

## Legacy Cartridge Deployment API - POST /deploy - Use the /cartridges APIs instead

In [ ]:
# Deploy a cartridge
try:
    f = open(CARTRIDGE_ZIP, "rb")
    resp = service.deploy_cartridge(archive_file=f, archive_file_content_type=CARTRIDGES_CONTENT_TYPE, update=True)
    print(resp.status_code)
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

---
<a id='user_data'></a>

# Delete User Data

In [ ]:
# Delete all custom tenant data
try:
    resp = service.delete_user_specific_artifacts()
    print("Response Code:",resp.status_code)
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)